In [1]:
%load_ext nb_black

import warnings

warnings.simplefilter(action="ignore", category=FutureWarning)

import pymc3 as pm
import pandas as pd
import numpy as np
import seaborn as sns
import arviz as az
import theano
import theano.tensor as tt
import matplotlib.pyplot as plt
import statsmodels.stats.api as sms
import altair as alt
import json
import gzip, pickle


WARNING (theano.tensor.blas): Using NumPy C-API based implementation for BLAS functions.


<IPython.core.display.Javascript object>

In [2]:
model_data = pd.read_table("./data/update_simple.tab", sep="\t", index_col=0)
model_data["count_scaled"] = model_data["count"] / 10000
model_data

,count,day,week,count_scaled
0,158016,Nov 25,1,15.8016
1,218733,Dec 2,2,21.8733
2,268026,Dec 9,3,26.8026
3,311895,Dec 16,4,31.1895
4,358098,Dec 23,5,35.8098
...,...,...,...,...
138,2406672,Jul 18,139,240.6672
139,2413273,Jul 25,140,241.3273
140,2419440,Aug 01,141,241.9440
141,2425048,Aug 08,142,242.5048


<IPython.core.display.Javascript object>

In [3]:
len_observed = len(model_data)

sets = ["CotA", "AoA", "WC", "MM", "DT"]
week_of_release = [0, 27, 50, 85, 126]

with pm.Model() as model_6:
    # priors
    sigma = pm.Exponential("sigma", lam=1.0)  # Sigma for likelihood function

    # COVID started at different points in different countries, it should be around week 70 give or take a week or two
    covid_start = pm.Data("covid_start", 68)

    # We know from the previous that mu before and during covid should be around 0.8 and 3.0 respectively
    # Sigma is reduced here not to diverge to far from these values
    BoundNormal_0 = pm.Bound(pm.Normal, lower=0)
    weekly_registrations_covid = BoundNormal_0(
        "weekly_registrations_covid", mu=0.8, sigma=0.5
    )
    weekly_registrations_precovid = BoundNormal_0(
        "weekly_registrations_precovid", mu=3, sigma=0.5
    )

    weekly_registrations_base = pm.math.switch(
        covid_start >= model_data.week,
        weekly_registrations_precovid,
        weekly_registrations_covid,
    )

    # Model extra registrations due to shifting interest (like new sets being released)
    # The interest factor is calculated on a weekly basis
    decay_factor = pm.Exponential("decay_factor", lam=1.0)
    interest = [pm.HalfNormal(f"{s}_interest", sigma=2) for s in sets]

    interest_decayed = [interest[0]]
    for i in range(len_observed - 1):
        new_element = interest_decayed[i] * decay_factor

        for ix, wor in enumerate(week_of_release[1:], start=1):
            if i == wor:
                new_element += interest[ix]

        interest_decayed.append(new_element)

    # there were 150k decks registered the first week, that is the initial value
    y0 = tt.zeros(len_observed)
    y0 = tt.set_subtensor(y0[0], 15)

    outputs, _ = theano.scan(
        fn=lambda t, y, ws, intfac: tt.set_subtensor(
            y[t], (ws[t] + weekly_registrations_precovid * intfac[t]) + y[t - 1]
        ),
        sequences=[tt.arange(1, len_observed)],
        outputs_info=y0,
        non_sequences=[weekly_registrations_base, interest_decayed],
        n_steps=len_observed - 1,
    )

    total_registrations = pm.Deterministic("total_registrations", outputs[-1])
    sales_delta = pm.Deterministic(
        "sales_difference",
        np.log2(weekly_registrations_covid / weekly_registrations_precovid),
    )

    # Likelihood
    likelihood = pm.Normal(
        "y", mu=total_registrations, sigma=sigma, observed=model_data.count_scaled
    )



C:\Users\Sebastian\.conda\envs\pymc3\lib\site-packages\theano\gpuarray\dnn.py:192: UserWarning: Your cuDNN version is more recent than Theano. If you encounter problems, try updating Theano or downgrading cuDNN to a version >= v5 and <= v7.
  warnings.warn(


<IPython.core.display.Javascript object>

In [ ]:
with model_6:
    # posterior
    trace_6 = pm.sample(1000, cores=10, chains=4, init="adapt_diag")

In [ ]:
def pickle_model(output_path: str, model, trace):
    """Pickles PyMC3 model and trace"""
    with gzip.GzipFile(output_path, "wb") as file:
        file.write(pickle.dumps({"model": model, "trace": trace}))


pickle_model(
    "./models/model_6.pickle.gz",
    model_6,
    trace_6,
)

In [4]:
def read_pickle(input_path: str):
    with gzip.GzipFile(input_path, "rb") as file:
        data = file.read()
        unpacked_data = pickle.loads(data)

    return unpacked_data["model"], unpacked_data["trace"]

model_6, trace_6 = read_pickle("./models/model_6.pickle.gz")


<IPython.core.display.Javascript object>

In [5]:
from pymc3_altair import plot_fit_altair

with model_6:
    pm.set_data({"covid_start": 68})

chart = plot_fit_altair(
    model_6,
    trace_6,
    model_data.rename(columns={"week": "Week_nr", "count_scaled": "Total_scaled"}),
    x_range=(1, max(model_data.week)),
)
chart.properties(width="container").save("./altair_output/model_6_covid.json")
chart

alt.LayerChart(...)

<IPython.core.display.Javascript object>

In [6]:
with model_6:
    pm.set_data({"covid_start": 500})

chart_without_covid = plot_fit_altair(
    model_6,
    trace_6,
    model_data.rename(columns={"week": "Week_nr", "count_scaled": "Total_scaled"}),
    x_range=(1, max(model_data.week)),
)
chart_without_covid.properties(width="container").save(
    "./altair_output/model_6_no_covid.json"
)
chart_without_covid

alt.LayerChart(...)

<IPython.core.display.Javascript object>

In [7]:
with model_6:
    sales_baseline_pp = pm.sample_posterior_predictive(
        trace_6,
        var_names=[
            "weekly_registrations_precovid",
            "weekly_registrations_covid",
            "sales_difference",
        ],
        random_seed=873,
    )
sales_baseline_df = pd.DataFrame(sales_baseline_pp).rename(
    columns={
        "weekly_registrations_precovid": "Pre-COVID",
        "weekly_registrations_covid": "COVID",
    }
)
sales_baseline_df

,Pre-COVID,COVID,sales_difference
0,1.553971,0.634816,-1.291549
1,1.542812,0.645240,-1.257655
2,1.541580,0.643080,-1.261339
3,1.631539,0.610476,-1.418227
4,1.626744,0.602051,-1.434029
...,...,...,...
3995,1.571249,0.607847,-1.370132
3996,1.567531,0.604693,-1.374220
3997,1.592271,0.620149,-1.360399
3998,1.596403,0.643764,-1.310221


<IPython.core.display.Javascript object>

In [8]:
weekly_registrations = (
    alt.Chart(sales_baseline_df)
    .transform_fold(
        list(sales_baseline_df.columns)[:2],
        as_=["Period", "value"],
    )
    .transform_density(
        density="value", bandwidth=0.3, groupby=["Period"], extent=[0, 3]
    )
    .mark_area(opacity=0.5)
    .encode(
        alt.X("value:Q", title="# Registered Decks Weekly (x10 000)"),
        alt.Y("density:Q", title=""),
        alt.Color("Period:N"),
        tooltip=["Period:N"],
    )
)
weekly_registrations.properties(width="container").save(
    "./altair_output/model_6_weekly_registrations.json"
)
weekly_registrations.properties(width=300, height=200)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [9]:
alt.Chart(sales_baseline_df).transform_density(
    density="sales_difference", bandwidth=0.3, extent=[-3, 3]
).mark_area(opacity=0.5).encode(
    alt.X("value:Q", title="Difference in sales (log transformed)"),
    alt.Y("density:Q", title=""),
).properties(
    width=300, height=200
)

alt.Chart(...)

<IPython.core.display.Javascript object>

In [10]:
color_range_ = ["#ad2117", "#0f69bd", "#940fbd", "#322d3b", "#5bbab9"]
set_names = [f"{s}_interest" for s in sets]

with model_6:
    set_interest_pp = pm.sample_posterior_predictive(
        trace_6,
        var_names=set_names,
        random_seed=873,
    )
set_interest_df = pd.DataFrame(set_interest_pp).rename(
    columns={a: a.split("_")[0] for a in set_names}
)
set_interest_df

,CotA,AoA,WC,MM,DT
0,4.283234,1.848067,0.989880,1.249845,1.069137
1,4.324696,1.767902,0.949493,1.269354,1.019132
2,4.283745,1.874406,1.008454,1.221570,1.027482
3,4.087993,1.556534,0.782344,1.425270,1.181501
4,4.147560,1.540137,0.789467,1.428455,1.170471
...,...,...,...,...,...
3995,4.107419,1.623658,0.934383,1.311517,1.133561
3996,4.129786,1.732192,0.880380,1.369333,1.166110
3997,4.232824,1.657795,0.937484,1.342115,1.128827
3998,4.325662,1.790848,0.861446,1.284408,1.065495


<IPython.core.display.Javascript object>

In [11]:
interest_plot = (
    alt.Chart(set_interest_df)
    .transform_fold(
        list(set_interest_df.columns),
        as_=["Set", "value"],
    )
    .transform_density(density="value", bandwidth=0.3, groupby=["Set"], extent=[0, 6])
    .mark_area(opacity=0.7)
    .encode(
        alt.X("value:Q", title="Popularity"),
        alt.Y("density:Q", title=""),
        alt.Color(
            "Set:N",
            sort=list(set_interest_df.columns),
            scale=alt.Scale(domain=list(set_interest_df.columns), range=color_range_),
        ),
        tooltip=["Set:N"],
    )
)
interest_plot.properties(width="container").save(
    "./altair_output/model_6_set_interest.json"
)

interest_plot.properties(width=300, height=200)

alt.Chart(...)

<IPython.core.display.Javascript object>